# Metrics and Losses

DRYTorch helps you standardize and document your experiment's metrics and loss.

### Terminology

A **metric** describes and assesses the performance of a model given its outputs and provided targets (i.e. the labels in supervised learning). A **loss** is used for optimization and has stricter requirements (e.g., differentiability), but it can also function as a metric. We refer to both as an **objective** and reserve the term 'metric' for objectives not used for parameter optimization.



## Compatibility with existing libraries
DRYTorch does not provide implementations of common losses. Instead, it encourages the use of implementations from other popular libraries by defining protocols that ensure full support to the relative classes.

### For Validating and Testing
The Objective protocol has been design to be compatible with the Metric classes from [torchmetric](https://lightning.ai/docs/torchmetrics/stable/) and [TorchEval](https://docs.pytorch.org/torcheval/stable/). You can use these metrics' instances directly when constructing a DRYTorch validation or test class.

### For Training
Some metrics from torchmetrics can also be used as losses. The Loss protocol accepts those metrics assuming they are differentiable. You therefore use it for building a DRYTorch training class.

In [ ]:
import torch
import torchmetrics

from torcheval import metrics as eval_metrics

from drytorch import protocols as p


torch_metric = torchmetrics.MeanSquaredError()
eval_metric = eval_metrics.MeanSquaredError()


def supported_for_validation(
    metric: p.ObjectiveProtocol[torch.Tensor, torch.Tensor],
) -> bool:
    """Test metric follows the Objective protocol."""
    return isinstance(metric, p.ObjectiveProtocol)


supported_for_validation(eval_metric) and supported_for_validation(torch_metric)

True

In [ ]:

def supported_for_training(
    metric: p.LossProtocol[torch.Tensor, torch.Tensor],
) -> bool:
    """Test metric follows the Loss protocol."""
    return isinstance(metric, p.LossProtocol)


supported_for_training(torch_metric)

True

## DRYTorch classes
DRYTorch objective classes are wrappers around user-defined metric and loss functions. These classes accept all function signatures, as long as they return a tensor from Torch. The class is responsible to call, document and aggregate the values of the functions. Furthermore, they are easily composable.


### The Metric and MetricCollection classes
The Metric class from DRYTorch is a callable that is generic on the outputs and targets arguments. 